## Calculate Some Morphological Parameters of the Basin using GRASS GIS

In [1]:
import os
import sys
import subprocess
import shutil
import binascii
import tempfile

from grass_session import Session
import grass.script as gs
import grass.script.setup as gsetup

import numpy as np
import matplotlib
import wx
import math
import csv

In [2]:
os.chdir('/home/shirobakaidou/eagle/MET_Spatial_Python/grass')
os.getcwd()

'/home/shirobakaidou/eagle/MET_Spatial_Python/grass'

In [3]:
# Define GRASS Database
dbase = os.getcwd()
location = 'mylocation'

# Set GISBASE environment variable
gisbase = '/usr/lib/grass78'
os.environ['GISBASE'] = str(gisbase)

# Initialize
gsetup.init(os.environ['GISBASE'], dbase, location, 'PERMANENT')

print(gs.gisenv())

{'GISDBASE': '/home/shirobakaidou/eagle/MET_Spatial_Python/grass', 'LOCATION_NAME': 'mylocation', 'MAPSET': 'PERMANENT'}


In [4]:
# Create Location
gs.create_location(dbase, location)

In [20]:
# Create a new location, using a georeferenced file
#gs.run_command('g.proj', flags="c",
#               georef="/home/shirobakaidou/eagle/MET_Spatial_Python/data/hydroDEM_clip.tif",
#               location="rbasin_reproduce")

Location <rbasin_reproduce> created
You can switch to the new location by
`g.mapset mapset=PERMANENT location=rbasin_reproduce`


0

In [5]:
# Switch to the location_vietnam, mapset "PERMANENT"
gs.run_command('g.mapset', 
               #location="mylocation",
               location="rbasin_reproduce",
               mapset="PERMANENT")

Mapset switched. Your shell continues to use the history for the old mapset
You can switch the history by commands:
history -w; history -r
/home/shirobakaidou/eagle/MET_Spatial_Python/grass/rbasin_reproduce/PERMANENT/.bash_history;
HISTFILE=/home/shirobakaidou/eagle/MET_Spatial_Python/grass/rbasin_reproduce/PERMANENT/.bash_history


0

In [ ]:
# Install hydrology related GRASS Addons

#gs.run_command('g.extension', extension='r.basin', operation='add')
#gs.run_command('g.extension', extension='r.hypso', operation='add')
#gs.run_command('g.extension', extension='r.stream.basins', operation='add')
#gs.run_command('g.extension', extension='r.stream.distance', operation='add')
#gs.run_command('g.extension', extension='r.stream.order', operation='add')
#gs.run_command('g.extension', extension='r.stream.snap', operation='add')
#gs.run_command('g.extension', extension='r.stream.stats', operation='add')
#gs.run_command('g.extension', extension='r.width.funct', operation='add')
#gs.run_command('g.extension', extension='r.stream.channel', operation='add')
#gs.run_command('g.extension', extension='r.stream.segment', operation='add')
#gs.run_command('g.extension', extension='r.stream.slope', operation='add')

In [6]:
# Display CRS Info
gs.run_command('g.proj', flags="p")

-PROJ_INFO-------------------------------------------------
name       : WGS 84 / UTM zone 48N
datum      : wgs84
ellps      : wgs84
proj       : utm
zone       : 48
no_defs    : defined
-PROJ_EPSG-------------------------------------------------
epsg       : 32648
-PROJ_UNITS------------------------------------------------
unit       : meter
units      : meters
meters     : 1


0

In [41]:
# Read GeoTiFF as GRASS Raster (Import Raster)
gs.run_command('r.in.gdal', 
               #flags='e', # update the default region
               input='/home/shirobakaidou/eagle/MET_Spatial_Python/data/hydroDEM_clip.tif',
               output='r_elevation',
               quiet=True,
               overwrite=True)

0

In [8]:
# Set Region
gs.run_command('g.region', 
               flags="pa",
               raster='r_elevation')

projection: 1 (UTM)
zone:       48
datum:      wgs84
ellipsoid:  wgs84
north:      1998522.28534274
south:      1937087.40340192
west:       464222.85466713
east:       513265.29261131
nsres:      87.88967374
ewres:      87.88967374
rows:       699
cols:       558
cells:      390042


0

In [9]:
# Define Threshold:
# source: https://github.com/OSGeo/grass-addons/blob/master/grass7/raster/r.basin/r.basin.py
resolution = gs.region()['nsres']
th = 1000000 / (resolution**2)
gs.message( "threshold : %s" % th )

threshold : 129.45662942175616


In [10]:
# Define Output Location
outpath = os.path.join(os.path.split(os.getcwd())[0], 'output', 'location_reproduce_rbasin')

In [49]:
# Watershed SFD (single flow direction)
gs.run_command('r.watershed',
               flags='ams',
               elevation='r_elevation',
               accumulation='r_accumulation', # output accumulation
               drainage='r_drainage', # output flow direction
               convergence=5,
               quiet=True,
               overwrite=True)
gs.message("Flow Direction map and Flow Accumulation map done.")

Flow Direction map and Flow Accumulation map done.


In [48]:
# Stream Extraction
gs.run_command('r.stream.extract', 
               elevation = 'r_elevation',
               accumulation = 'r_accumulation',
               threshold = th,
               #d8cut =  1000000000,
               mexp = 0,
               stream_rast = 'r_stream_e', # output unique stream ids
               direction = 'r_drainage_e',
               quiet=True,
               overwrite=True)
gs.message("Stream Extraction done.")

Stream Extraction done.


In [81]:
# Delineation of basin: Create outlet
# Create outlet
gs.write_command('v.in.ascii',
                 input='-',
                 output='v_outlet',
                 sep=",",
                 stdin="%s,9999" % ("490282.15625,1966838"), # <modify coordinates>
                 quiet=True,
                 overwrite=True)
# The point is snapped to the nearest point which lies on the streamline
gs.run_command('r.stream.snap',
               input='v_outlet', 
               stream_rast='r_stream_e', # Input stream network
               output='v_outlet_snap', # output vector points map
               quiet=True,
               overwrite=True)
gs.run_command('v.to.rast',
               input='v_outlet_snap',
               output='r_outlet',
               use='cat',
               type='point',
               layer=1,
               value=1,
               quiet=True,
               overwrite=True)
# Delineates basins
gs.run_command('r.stream.basins',
               direction='r_drainage_e', # Input flow direction
               points='v_outlet_snap', # Input vector points
               basins='r_basin', # Output basin
               quiet=True,
               overwrite=True)
gs.message("Delineation of basin done")

Delineation of basin done


In [50]:
# Creation of Slope and Aspect maps
gs.run_command('r.slope.aspect',
               elevation='r_elevation',
               slope='r_slope',
               aspect = 'r_aspect',
               quiet=True,
               overwrite=True)
gs.message("Slope and Aspect maps done.")

Slope and Aspect maps done.


In [51]:
# Create Basin Mask (Vector)
gs.run_command('r.to.vect',
               input='r_basin',
               output='v_basin',
               type='area',
               flags='sv',
               quiet=True,
               overwrite=True)
gs.message("Creation of Basin mask done.")

Creation of Basin mask done.


In [54]:
# Add two columns to the table 'basin': Area and Perimeter
gs.run_command('v.db.addcolumn',
               map='v_basin',
               columns='area double precision') # name & type
gs.run_command('v.db.addcolumn',
               map='v_basin',
               columns='perimeter double precision')
gs.message("Column 'Area' and Column 'Perimeter' are added \n to the attribute table of basin")

ERROR: Column <area> is already in the table. Skipping.
ERROR: Column <perimeter> is already in the table. Skipping.
Column 'Area' and Column 'Perimeter' are added
to the attribute table of basin


In [56]:
# Populate Perimeter Column
gs.run_command('v.to.db',
               map='v_basin',
               type='line,boundary',
               layer=1,
               qlayer=1,
               option='perimeter',
               units='kilometers',
               columns='perimeter',
               quiet=True,
               overwrite=True)
gs.run_command('v.to.db',
               map='v_basin',
               type='line,boundary',
               layer=1,
               qlayer=1,
               option='area',
               units='kilometers',
               columns='area',
               #flags='c',
               quiet=True,
               overwrite=True)
gs.message("Column 'Area' and 'Perimeter' are filled with value")

Column 'Area' and 'Perimeter' are filled with value


In [59]:
# Export Selected Attributes
gs.run_command('v.db.select',
               map='v_basin',
               column='perimeter, area',
               file=os.path.join(outpath,'basin_attr'),
               overwrite=True)
gs.message("The attribute of area and perimeter are exported.")

The attribute of area and perimeter are exported.


In [60]:
# Read Perimeter and Area from the text file
tmp = open(os.path.join(outpath,"basin_attr"), "r")
tmp = tmp.read()
tmp
#tmp.read().split('\n')

'perimeter|area\n111.389201|263.591175\n'

In [68]:
# Extract Perimeter and Area
perimeter_basin = float(tmp.split('\n')[1].split('|')[0])
area_basin = float(tmp.split('\n')[1].split('|')[1])
print("The perimeter of the basin is",perimeter_basin, "km; \n","The area of the basin is", area_basin, "km2") # both km

The perimeter of the basin is 111.389201 km; 
 The area of the basin is 263.591175 km2


In [69]:
# Create stream order maps: strahler, horton, hack, shreeve
gs.run_command('r.stream.order',
               stream_rast = 'r_stream_e',
               direction = 'r_drainage_e',
               strahler = 'r_strahler', # output Strahler order
               shreve = 'r_shreve', # output Shreve order
               horton = 'r_horton', # output Horton order
               hack = 'r_hack', # output Hack order
               quiet=True,
               overwrite=True)
gs.message("Stream order maps of Strahler, Horton, Hack and Shreeve are created.")

Stream order maps of Strahler, Horton, Hack and Shreeve are created.


In [70]:
# Calculate Average Slope and Slope Ratio
stream_stats = gs.read_command('r.stream.stats',
               stream_rast='r_horton',
               direction='r_drainage_e',
               elevation='r_elevation',
               quiet=True)
print("Calculated stream statistics incl. averge slope and slope ratio: \n",
      stream_stats)

Calculated stream statistics incl. averge slope and slope ratio: 
 
Summary:
Max order | Tot.N.str. | Tot.str.len. | Tot.area. | Dr.dens. | Str.freq. 
  (num)   |    (num)   |     (km)     |   (km2)   | (km/km2) | (num/km2) 
        5 |        251 |     711.1800 | 1161.7713 |   0.6122 |  0.2160 

Stream ratios based on regression coefficient:
 Bif.rt. | Len.rt. | Area.rt. | Slo.rt. | Grd.rt. 
  3.3896 |  2.3988 |   3.6128 |  1.6948 |  1.7338

Averaged stream ratios with standard deviations:
 Bif.rt. | Len.rt. | Area.rt. | Slo.rt. | Grd.rt. 
  3.7244 |  2.6211 |   2.8147 |  1.6919 |  1.7265
  1.9316 |  0.8282 |   2.0141 |  0.3944 |  0.3134

Order | Avg.len |  Avg.ar  |  Avg.sl |  Avg.grad. | Avg.el.dif
 num  |   (km)  |  (km2)   |  (m/m)  |    (m/m)   |     (m)   
    1 |  1.2834 |   2.6950 |  0.0879 |     0.0730 | 95.5803
    2 |  4.4655 |  10.7632 |  0.0506 |     0.0476 | 198.7209
    3 | 12.1847 |  48.4767 |  0.0328 |     0.0307 | 330.7273
    4 | 18.1431 | 133.8441 |  0.0148 |     0

In [40]:
#gs.run_command('db.out.ogr',
#               input='stream_stats' ,
#               output=os.path.join(outpath,'stream_stats.csv'),
#               format='CSV')
#stream_stats.split('\n')

In [73]:
# Distance to Outlet
gs.run_command('r.stream.distance',
               stream_rast = 'r_outlet',
               direction = 'r_drainage_e',
               flags = 'o',
               distance = 'r_distance',
               quiet=True,
               overwrite=True)
gs.message("Distance to Outlet is calculated.")

Distance to Outlet is calculated.


In [75]:
# Calc Mean Slope
#baricenter_slope_baricenter = gs.read_command("r.volume",
#                                              input='r_slope',
#                                              clump='r_basin')
#mean_slope = baricenter_slope_baricenter[5]
#print(mean_slope)
#print(baricenter_slope_baricenter)
#baricenter_slope_baricenter#.split('\n')[]

In [74]:
# Circularity Ratio
circularity_ratio = (4*math.pi*area_basin)/(perimeter_basin**2)
print("Circularity Ratio is", circularity_ratio)

Circularity Ratio is 0.26696513826844187


In [78]:
# Main Channel
gs.mapcalc ("$r_mainchannel = if($r_hack==1,1,null())",
            r_hack = 'r_hack', # input
            r_mainchannel = 'r_mainchannel', # output
            quiet=True,
            overwrite=True) 

gs.run_command("r.thin", 
               input = 'r_mainchannel',
               output = 'r_mainchannel'+'_thin',
               quiet=True,
               overwrite=True)

gs.run_command('r.to.vect',
               input='r_mainchannel'+'_thin',
               output='v_mainchannel',
               type='line',
               #verbose=True,
               quiet=True,
               overwrite=True)
gs.message("Main Channel is extracted.")

Main Channel is extracted.


In [82]:
# Get coordinates of the outlet (belonging to stream network)
gs.run_command('v.db.addtable', map='v_outlet_snap',
               quiet=True,
               overwrite=True)

gs.run_command('v.db.addcolumn',
               map='v_outlet_snap',
               columns='x double precision, y double precision',
               quiet=True,
               overwrite=True)

gs.run_command('v.to.db',
               map='v_outlet_snap',
               option='coor',
               col='x,y',
               quiet=True,
               overwrite=True)

gs.run_command('v.out.ascii',
               input='v_outlet_snap',
               output=os.path.join(outpath,'outlet_coords.txt'),
               cats=1,
               format="point",
               quiet=True,
               overwrite=True)

gs.message("Coordinate of Outlet is exported")

0

In [84]:
outlet_coords = open(os.path.join(outpath,'outlet_coords.txt')).readline()
east_o, north_o, cat = outlet_coords.split('|')
print("The east and north coordinateof outlet is", east_o, ",", north_o)

The east and north coordinateof outlet is 490282.15625 , 1966838


In [85]:
# Basin Length
param_mainchannel = gs.read_command('v.what',
                                    map = 'v_mainchannel',
                                    coordinates='%s,%s' % (east_o, north_o),
                                    distance=5)
basin_length = float(param_mainchannel.split('\n')[7].split()[1])/1000 # km

print("The Basin length is", basin_length, "km.")

The Basin length is 75.032720702 km.


In [86]:
# Elongation Ratio
# 1. Interpreted from Paper
elon_ratio1 = perimeter_basin/(math.pi*basin_length)

# 2. Based on r.basin
elon_ratio2 = (2*math.sqrt(area_basin/math.pi))/basin_length

print("The elongation ratio is")
print(elon_ratio1, elon_ratio2)

The elongation ratio is
0.4725442921526407 0.24415734740038567


In [87]:
# Form Factor
form_factor = area_basin/(basin_length**2)
print("The Form Factor is ", form_factor)

The Form Factor is  0.046819791716406545


In [89]:
# Relative Relief (Hmax - Hmin)
height_attr = gs.read_command('r.info', 
                flags='r',
                map='r_elevation')
height_min = float(height_attr.strip().split('\n')[0].split('=')[-1])
height_max = float(height_attr.strip().split('\n')[1].split('=')[-1])
print("The max Altitude is ", height_min, "m; \n", 
      "The min Altitude is ", height_max, "m.")

relative_relief = height_max - height_min
print("The relative Relief is ", relative_relief, "m.") # meter

The max Altitude is  165.0 m; 
 The min Altitude is  1532.0 m.
The relative Relief is  1367.0 m.


In [90]:
# Relief Ratio
relief_ratio = (relative_relief/1000)/basin_length
print("The Relief Ratio is ", relief_ratio)

The Relief Ratio is  0.018218718276645972


In [91]:
# Dissection Index
dissection_index = relative_relief/height_max
print("The Dissection Index is ", dissection_index)

The Dissection Index is  0.8922976501305483


In [92]:
# Hypometric Curve
gs.run_command('r.hypso', 
               map = 'r_elevation',
               image=os.path.join(outpath,'hysometric_curve.png'),
               flags='a',
               quiet=True,
               overwrite=True)
gs.message("The Hupometric Curve is generated.")

Gtk-Message: 16:26:13.559: Failed to load module "atk-bridge"
Gtk-Message: 16:26:13.590: Failed to load module "canberra-gtk-module"


Tot. cells 158241.0
Hypsometric | quantiles
823 | 0.025
755 | 0.05
693 | 0.1
582 | 0.25
436 | 0.5
250 | 0.75
196 | 0.9
176 | 0.975


Done!


The Hupometric Curve is generated.


In [94]:
# Write Indices to CSV File
csvfile = os.path.join(outpath, "output_indices.csv")
with open(csvfile, 'w') as f:
    writer = csv.writer(f)
    writer.writerow(['Relative Relief (m)'] + [relative_relief])
    writer.writerow(['Relief Ratio'] + [relief_ratio])
    writer.writerow(['Dessection Index'] + [dissection_index])
    writer.writerow(['Form factor'] + [form_factor])
    writer.writerow(['Circularity Ratio'] + [circularity_ratio])
    writer.writerow(['Elongation Ratio'] + [elon_ratio1])
    writer.writerow(['Basin Area (km2)'] + [area_basin])
    writer.writerow(['Basin Perimeter (km)'] + [perimeter_basin])
    
gs.message("The indices are summarised and exported.")    

The indices are summarised and exported.
